# Technical Report

## Introduction

This report details the steps taken to transform raw data on NHS prescription costs into data suitable for analysis. It will also show how the analysis was performed and how outputs are produced (tables / visualisations etc). For details on the need for this analysis, please see the project summary.

All work was done using the Python programming language. This report will include the code used to produce the analysis. This code can be run if the reader is viewing this report on github to produce the same results.

## Merging the Data

The raw data is split into 6 different files, for each of the months to be analysed (6 months from July to December).

The first step in analysing the data was therefore to join the datasets together, and add a column indicating which month the data was for. This was necessary as the specification requested a analysis of trends over time, so it was important to know which timeframe data had come from.

This was achieved through the following functions:

### Creating a dictionary of filepaths

Firstly, data for each month needs to be read into Python from the raw .csv files. It is important to know which month each file relates to, so this was done by creating a dictionary with the month as the key and a pandas dataframe of the relevant data as the value.

In [6]:
import pandas as pd
import os

#global variables
FILEPATH_IN = "../Data/raw_data"
FILEPATH_OUT = "../Data/processed_data"

#create dictionary of all month's data

def create_data_dict(filepath):
    data_dict = {}

    for folder in os.listdir(filepath):
        for filename in os.listdir(filepath + '/' +folder):
            data_dict[folder[:folder.find('_')]] = pd.read_csv(filepath + '/' +folder+ '/' +filename, index_col=None, header=0)
    
    return data_dict

data_dict = create_data_dict(FILEPATH_IN)

c:\users\thoma\onedrive\documents\synoptic_project_qa\.qa_project\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


### Testing Column Continuity

The plan is to append the data by column. In order to do this, all the datasets need to have the same columns to ensure there are no errors. The following function was written to test this before joining the datasets together.

In [8]:
def test_cols_same(default_month, data_dict):

    july_col = list(data_dict['July'].columns)

    columns_dict = {}

    for key, values in data_dict.items():
        month_col = list(values.columns)
        incon_col =[col for col in month_col if col not in july_col]
        if len(incon_col) > 0:
            columns_dict[key] = incon_col
            
    return columns_dict

same_cols = test_cols_same("July",data_dict)

# if len(same_cols) > 0:
#     raise Exception as ValueError(same_cols)

July's dataset is used to compare to all the other data. If any of the other datasets have different columns to July, those columns are added to a dictionary and an error is raised detailing which dataset contains the differing column. Fortunately for the initial six datasets, all columns are the same. However, this function will be useful in future if additional data is added.

### Merging the Data into one Dataframe

The data can now be joined together into one dataset, before cleaning the columns. This is achieved by iterating through the datasets in the data dictionary, adding a column indicating which month the data is for, adding the revised data to a list and then merging all data in that list into one dataframe. Finally, the data is written to the output folder specified in the global variables.

In [11]:
def join_datasets(data_dict):
    datasets = []

    for key, values in data_dict.items():
        values['Month'] = key
        datasets.append(values)
    
    full_dataset = pd.concat(datasets, axis=0, ignore_index=True)
    return full_dataset

full_data = join_datasets(data_dict)

full_data.to_csv(FILEPATH_OUT + "/" + "joined_data.csv")

full_data.head()

## Cleaning the Data

Now that the data is in one dataframe, the next step is to check that the columns contain consistent rows and that the data types for the columns make sense for further analysis.

In [12]:
full_data.dtypes 

Regional Office Name     object
Regional Office Code     object
Area Team Name           object
Area Team Code           object
PCO Name                 object
PCO Code                 object
Practice Name            object
Practice Code            object
BNF Code                 object
BNF Description          object
Items                     int64
Quantity                  int64
ADQ Usage               float64
NIC                     float64
Actual Cost             float64
Month                    object
dtype: object

In [13]:
full_data.head()

,Regional Office Name,Regional Office Code,Area Team Name,Area Team Code,PCO Name,PCO Code,Practice Name,Practice Code,BNF Code,BNF Description,Items,Quantity,ADQ Usage,NIC,Actual Cost,Month
0,LONDON,Y56,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,UNIDENTIFIED DOCTORS,07L998,0101021B0BEADAJ,Gaviscon Infant_Oral Pdr Sach,1,15,0.0,4.82,4.47711,Aug
1,LONDON,Y56,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,UNIDENTIFIED DOCTORS,07L998,0105010B0BBAFAS,Asacol_MR Tab E/C 800mg,1,168,84.0,109.80,101.71878,Aug
2,LONDON,Y56,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,UNIDENTIFIED DOCTORS,07L998,0301011R0AAAPAP,Salbutamol_Inha 100mcg (200 D) CFF,1,1,50.0,1.50,1.40183,Aug
3,LONDON,Y56,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,UNIDENTIFIED DOCTORS,07L998,0304010ABAAAAAA,Desloratadine_Tab 5mg,1,30,30.0,1.01,0.94795,Aug
4,LONDON,Y56,NORTH EAST LONDON AREA,Q61,BARKING & DAGENHAM CCG,07L00,UNIDENTIFIED DOCTORS,07L998,0304010D0AAAAAA,Loratadine_Tab 10mg,1,14,14.0,0.35,0.43660,Aug


In [14]:
full_data['Area Team Name'].unique()

array(['NORTH EAST LONDON AREA', 'NORTH WEST LONDON AREA',
       'SOUTH LONDON AREA'], dtype=object)

## Analysing the Data

Now that the data set is cleaned and reduced to the required columns, the data can be analysed to draw out insights. The focus will first be on descriptive statistics to meet the customer's requirements: 

- Trends in costs by Area Team.
- Trends in costs by PCO.
- Trends in costs by GP Practice.
- GP Practices with highest prescription costs.
- GP Practices with highest quantity of prescription.
- Comparing practices with both high cost and high prescription quantity.

There will then be an exploration of data modelling to determine if any further insight can be found.

## Trends by Area Team

The 'Area Team' is a large geographical area. For this data, we are limited to three areas within London - North East, North West and South. The following code produces a chart that shows the trend over time for these areas.

In [26]:
## code here
import numpy as np

data = full_data[['Area Team Name', 'Month', 'Actual Cost']]

# area_team_data = data.groupby(['Area Team Name','Month']).sum()

table = pd.pivot_table(data, values='Actual Cost', index=['Area Team Name'],
                    columns=['Month'], aggfunc=np.sum)

table

## months are in incorrect order

Month,Aug,Dec,July,Nov,Oct,Sep
Area Team Name,,,,,,
NORTH EAST LONDON AREA,3.205441e+07,3.212212e+07,3.428795e+07,3.370686e+07,3.448605e+07,3.324981e+07
NORTH WEST LONDON AREA,1.944534e+07,1.963164e+07,2.079511e+07,2.050573e+07,2.107448e+07,2.012940e+07
SOUTH LONDON AREA,3.081437e+07,3.028186e+07,3.234377e+07,3.182524e+07,3.277002e+07,3.129412e+07


In [2]:
import plotly.express as px

wide_df = table

fig = px.bar(wide_df, x="Month", y='Area Team Name', title="Prescription Costs by Area")
fig.show()

NameError: name 'table' is not defined